## HH парсер

### условия:
* санкт-петербург
* другие города, если удаленка
* python, sql, bi в описании требований
* вакансии за последний день
* ранжирование вакансий по требованиям

In [3]:
import requests, json, os
import pandas as pd
import numpy as np
import warnings
from datetime import datetime as dt
warnings.filterwarnings('ignore')
import webbrowser
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict
from tqdm import tqdm_notebook
from datetime import datetime as dt
import time
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('seaborn-ticks')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from bs4 import BeautifulSoup 
from tqdm import tqdm_notebook
from wordcloud import WordCloud, STOPWORDS

In [4]:
def collect_hh_vacancies(area, query, pages):    
    #######################################################################################
    def getPage(area, page, query):
        """
        Создаем метод для получения страницы со списком вакансий.
        Аргументы:
            page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
        """

        # Справочник для параметров GET-запроса
        params = {
            'text': f'NAME:{query}', 
            'area': f'{area}', 
            'page': page, 
            'per_page': 100
            }
        req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
        data = req.content.decode() 
        req.close()
        return data

    def flatten_json(json_file):
        L= []
        for k, v in i.items():
            if isinstance(v, dict):
                for k2, v2 in v.items():
                    if not(isinstance(v2, dict)):
                        L.append((f'{k}.{k2}', v2))
                    else:
                        for k3, v3 in v2.items():
                            L.append((f'{k}.{k2}.{k3}', v3))
                        
            else:
                L.append((k, v))        
        _df = pd.DataFrame.from_records(L).set_index(0)
        _df.columns = _df.loc['id'].values    
        return _df.iloc[1:].T  
    ###########################################################################################
    
    # список с коллекцией выгруженных вакансий
    L_rows=[]    

    for page_idx in tqdm_notebook(pages):

        # выгружаем
        json_spb = json.loads(getPage(area=area, page=page_idx,query=query))     
        
        try:
            
            for i in json_spb['items']: 
                
                # коллекционируем
                L_rows.append(flatten_json(i))
                
                # спим, чтобы сайт не ограничил доступ
                time.sleep(.1)   
        except:
            pass  
    
    # собираем датафрейм
    df = pd.concat(L_rows)
    for col in df.columns:
        try:
            df[col] = df[col].astype(np.float32)
        except:
            pass
    df['published_at'] = pd.to_datetime(df['published_at'])
    return df.sort_values('published_at', ascending = False).reset_index(drop = True)

In [5]:
df1 = collect_hh_vacancies(area=113,query='аналитик', pages=np.arange(10))
df2 = collect_hh_vacancies(area=113,query='data scientist', pages=np.arange(10))
df3 = collect_hh_vacancies(area=113,query='bi', pages=np.arange(10))

In [33]:
# собираем датафрейм
df = pd.concat([df1, df2, df3], 0).reset_index(drop = True).dropna(subset = ['snippet.requirement'])

# требования
req = df['snippet.requirement'].str.lower()
mask_spb = df['area.name'].isin(['Санкт-Петербург'])
mask_udalenka = df['schedule.name'] == 'Удаленная работа'
mask_python = req.apply(lambda x: 'python' in x)
mask_sql = req.apply(lambda x: 'sql' in x)
mask_bi = req.apply(lambda x: 'bi' in x)

# используем колонки
cols_to_use = ['name', 'published_at', 'salary.from', 'salary.to',\
               'employer.name', 'snippet.requirement',\
               'snippet.responsibility', 'schedule.name', 'alternate_url']
ru_cols_to_use=['название', 'дата_публикации', 'зп_от', 'зп_до', 'компания', 'требования',\
                'обязанности', 'график_работы', 'url']

# финальная таблица(вакансии за последний день)
df_final = df[(mask_spb|(~mask_spb&mask_udalenka)) & (mask_python|mask_sql|mask_bi)]\
               .reset_index(drop = True)\
               [cols_to_use]\
               .rename(columns = dict(zip(cols_to_use, ru_cols_to_use)))
df_final = df_final[df_final['зп_от'].apply(lambda x: np.isnan(x)|(x<100000))]

# дата публикации
datetimes = pd.to_datetime(df_final['дата_публикации'].dt.date)
# последняя публикация
date_today = pd.to_datetime(dt.now().date())
df_final = df_final[((date_today - datetimes) / np.timedelta64(1, 'D'))<=1]                   
df_final = df_final[~df_final.duplicated()]          

# ранжирование вакансий по требованиям
req = df_final['требования'].str.lower()
sql_mask = req.str.contains('sql').astype(int)
pandas_mask = req.str.contains('pandas').astype(int)
python_mask = req.str.contains('python').astype(int)
df_final = df_final.loc[(python_mask+pandas_mask+sql_mask).sort_values(ascending = False).index]

# выбрасываем middle и senior
mask_notmiddle = df_final['название'].str.lower().apply(lambda x: 'middle' not in x)
mask_notsenior = df_final['название'].str.lower().apply(lambda x: 'senior' not in x)
mask_notsenior_v2 = df_final['название'].str.lower().apply(lambda x: 'старший' not in x)
df_final = df_final[(mask_notmiddle)&(mask_notsenior)&(mask_notsenior_v2)].reset_index(drop = True)   


df_today = df_final[~df_final['компания'].isin(['Филберт', 'Softwise'])]
mask1 = (df_today['дата_публикации'].apply(lambda x: x.date()) >=pd.to_datetime(dt.now()))
mask2 = (df_today['дата_публикации'].apply(lambda x: x.hour) >= pd.to_datetime(dt.now()).hour)
df_absolutely_new = df_today[mask1&mask2]

In [34]:
df_today

,название,дата_публикации,зп_от,зп_до,компания,требования,обязанности,график_работы,url
0,Data Analyst,2021-06-19 12:58:15+03:00,20000.0,50000.0,MacBuildServer,Опыт работы с табличными данными. Навыки работ...,Изучать бизнес-данные и продумывать оптимальны...,Удаленная работа,https://hh.ru/vacancy/44904368
1,Аналитик BI,2021-06-19 21:00:42+03:00,NaN,NaN,ДИВАН.РУ,"Знания основ sql, python+pandas (будет большим...",Сбор требований и правила расчета ключевых пок...,Удаленная работа,https://hh.ru/vacancy/44494198
2,Data Scientist,2021-06-19 16:05:08+03:00,NaN,NaN,DIGINETICA,Уметь писать готовый к production код (Python/...,Разрабатывать модели схожести запросов и товар...,Удаленная работа,https://hh.ru/vacancy/44487032
3,Data Scientist,2021-06-19 17:50:13+03:00,NaN,NaN,ATI.SU,Знание SQL и умение писать сложные запросы. У ...,Сбор и анализ данных из различных источников. ...,Гибкий график,https://hh.ru/vacancy/45507190
4,Продуктовый аналитик,2021-06-19 15:21:55+03:00,NaN,NaN,"Лента, федеральная розничная сеть","Имеешь опыт работы в рознице, онлайн-проектах....",Проверять продуктовые гипотезы (проводить эксп...,Удаленная работа,https://hh.ru/vacancy/44913116
5,Аналитик,2021-06-19 18:23:05+03:00,NaN,NaN,Паблишер,"Понимание основ статистики, теории вероятносте...",Повышать качество принимаемых решений за счёт ...,Полный день,https://hh.ru/vacancy/45508070
6,Младший консультант Power BI,2021-06-19 08:05:54+03:00,NaN,NaN,"Северсталь, Москва",Опыт работы аналитиком BI от 2-х лет. Владение...,Проектирование и разработка визуализаций (Powe...,Полный день,https://hh.ru/vacancy/45085490
7,Web-аналитик,2021-06-19 10:49:47+03:00,NaN,NaN,Игроник,"SQL, Google BigQuery. Умение работать с API на...","Data-tracking (GTM, Google Analytics, Яндекс.М...",Удаленная работа,https://hh.ru/vacancy/44897436
8,Аналитик,2021-06-19 12:56:51+03:00,NaN,NaN,Колл Солюшенс,Знание SQL. Опыт проведения основных анализов ...,Анализ обращений клиентов и партнеров в разрез...,Полный день,https://hh.ru/vacancy/43815519
9,Продуктовый аналитик/ Data Analyst (junior),2021-06-19 17:20:45+03:00,NaN,NaN,3205.team,"понимать, моделировать и анализировать LTV, RO...","проверять гипотезы (формирование, тестирование...",Удаленная работа,https://hh.ru/vacancy/45505865


In [35]:
df_absolutely_new

,название,дата_публикации,зп_от,зп_до,компания,требования,обязанности,график_работы,url
14,Ведущий аналитик BI,2021-06-20 10:02:55+03:00,NaN,NaN,СИГМА,Опыт работы системным аналитиком не менее 3-х ...,Участие в проектах внедрения аналитических сис...,Полный день,https://hh.ru/vacancy/40888962
